In [19]:
import pandas as pd
import numpy as np
hosp = '/Users/mariama/Documents/GitHub/small-things/urgent_care_project/Hospitals.csv'
# Had to use R to reformat the CSV somehow?? I'm sure there was a better answer, but this worked.
hospital = pd.read_csv(hosp, encoding = "ISO-8859-1")
#del(hospital['Unnamed: 0']) #delete the numbered rows that R added in. 

pop_means = 'pop_means_tract.csv'
center = pd.read_csv(pop_means, encoding = "ISO-8859-1")

In [20]:
er = hospital[hospital.TYPE.isin(['GENERAL ACUTE CARE','CRITICAL ACCESS'])]

In [3]:
# Taken from: https://gist.github.com/rochacbruno/2883505 (Thanks!!)

# Haversine formula example in Python
# Author: Wayne Dyck

import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [4]:
hosp_latlon = pd.DataFrame(columns = ['latlon'])
hll = er[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

ct_latlon = pd.DataFrame(columns = ['latlon'])
ctll = center[['LATITUDE','LONGITUDE']].values.tolist()
ct_latlon['latlon'] = ctll

In [6]:
# Defining the function to get the closest hospitals
def get_nearest_hosp(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    #closest_hospital = er.iloc[temp.idxmin()]
    #closest_hospital = temp.min()
    closest_hospital_index = temp.idxmin()
    return closest_hospital_index

# Has to be two functions because .apply will only return one thing.
def get_nearest_hosp_dist(ct_locs):
    hosp_locs = hosp_latlon["latlon"]
    dist_list = []
    for item in hosp_locs: 
        dist_list.append(distance(item, ct_locs))
    temp = pd.DataFrame(dist_list)
    closest_hospital_dist = temp.min()
    return closest_hospital_dist

In [11]:
#How long will this take???
import time
start_time = time.time()

ct_ll  = ct_latlon.head()
ct_ll['latlon'].apply(get_nearest_hosp)
ct_ll['latlon'].apply(get_nearest_hosp_dist)

print((((len(ct_latlon)/5)*(time.time() - start_time))/60), "minutes")


32.22170651912689 minutes


In [ ]:
#Applies nearest hospital functions to add two new columns to center.
#center['hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
#center['dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)


In [ ]:
#Need to reindex ER, since earlier, I dropped out all non critical access hospitals.
#er = er.reset_index(drop=True)

# Merge population center & hospital lists together on the hospital index.
#fullset = pd.merge(center, er, left_on = "hosp_index", right_index=True, how = 'left')

#This is what it would look like without any hospitals having closed - probably a good approximation of 2011.

In [ ]:
#open_hosp = fullset[fullset.STATUS.isin(['OPEN'])]
#closed_hosp = fullset[fullset.STATUS.isin(['CLOSED'])]

In [23]:
# STAT2 is the status of hospitals that I checked, from a list of hospitals that had closed.
# Combining it with the STATUS that came in the original dataset, 
# to create a test column that aggregates the two.

er['test'] = np.where(((er['STATUS']=="CLOSED")|(er["STAT2"]=='CLOSED')), "CLOSED", "OPEN")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
# Getting the closest hospitals after removing closed hospitals

hosp_latlon = pd.DataFrame(columns = ['latlon'])

# looking for only open hospitals:
er_open = er[er.test.isin(['OPEN'])]
hll = er_open[['LAT','LON']].values.tolist()
hosp_latlon['latlon'] = hll

In [12]:
ct_current = center
ct_current['hosp_index'] = ct_latlon['latlon'].apply(get_nearest_hosp)
ct_current['dist'] = ct_latlon['latlon'].apply(get_nearest_hosp_dist)

In [13]:
ct_current.iloc[ct_current['dist'].idxmax()]

STATEFP           15.000000
COUNTYFP           3.000000
TRACTCE       981200.000000
POPULATION         0.000000
LATITUDE          27.784598
LONGITUDE       -175.860040
hosp_index      4162.000000
dist            1754.930087
Name: 20600, dtype: float64

In [25]:
#Need to reindex ER, since earlier, I dropped out all non critical access hospitals.
er_open = er_open.reset_index(drop=True)

# Merge population center & hospital lists together on the hospital index.
only_open = pd.merge(ct_current, er_open, left_on = "hosp_index", right_index=True, how = 'left')


In [28]:
only_open.to_csv("distance_2018.csv")